<a href="https://www.kaggle.com/code/rattans/road-accident-risk-xgboost?scriptVersionId=270308911" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Hello
**This is a beginner friendly intermediate level notebook ;)**

**Please do comment and spread your knowledge. Enlighten me and others.**

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e10/sample_submission.csv
/kaggle/input/playground-series-s5e10/train.csv
/kaggle/input/playground-series-s5e10/test.csv


In [2]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

# Importing the dataset

In [3]:
X_full = pd.read_csv('../input/playground-series-s5e10/train.csv')
X_test = pd.read_csv('../input/playground-series-s5e10/test.csv')

X_full.describe()

,id,num_lanes,curvature,speed_limit,num_reported_accidents,accident_risk
count,517754.000000,517754.000000,517754.000000,517754.000000,517754.000000,517754.000000
mean,258876.500000,2.491511,0.488719,46.112575,1.187970,0.352377
std,149462.849974,1.120434,0.272563,15.788521,0.895961,0.166417
min,0.000000,1.000000,0.000000,25.000000,0.000000,0.000000
25%,129438.250000,1.000000,0.260000,35.000000,1.000000,0.230000
50%,258876.500000,2.000000,0.510000,45.000000,1.000000,0.340000
75%,388314.750000,3.000000,0.710000,60.000000,2.000000,0.460000
max,517753.000000,4.000000,1.000000,70.000000,7.000000,1.000000


The two code blocks below are used to verify that the number of categorical values are same in both train and test set.

In [4]:
X_full.nunique()

id                        517754
road_type                      3
num_lanes                      4
curvature                    261
speed_limit                    5
lighting                       3
weather                        3
road_signs_present             2
public_road                    2
time_of_day                    3
holiday                        2
school_season                  2
num_reported_accidents         8
accident_risk                 98
dtype: int64

In [5]:
X_test.nunique()

id                        172585
road_type                      3
num_lanes                      4
curvature                    195
speed_limit                    5
lighting                       3
weather                        3
road_signs_present             2
public_road                    2
time_of_day                    3
holiday                        2
school_season                  2
num_reported_accidents         8
dtype: int64

# Features

**Here comes the most important feature**
* This is formula is taken from the original dataset

In [6]:
# Adding a base_risk feature

X_full['base_risk'] = (
        0.3 * X_full["curvature"] + 
        0.2 * (X_full["lighting"] == "night").astype(int) + 
        0.1 * (X_full["weather"] != "clear").astype(int) + 
        0.2 * (X_full["speed_limit"] >= 60).astype(int) + 
        0.1 * (np.array(X_full["num_reported_accidents"]) > 2).astype(int)
    )
X_test['base_risk'] = (
        0.3 * X_test["curvature"] + 
        0.2 * (X_test["lighting"] == "night").astype(int) + 
        0.1 * (X_test["weather"] != "clear").astype(int) + 
        0.2 * (X_test["speed_limit"] >= 60).astype(int) + 
        0.1 * (np.array(X_test["num_reported_accidents"]) > 2).astype(int)
)

In [7]:
X_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517754 entries, 0 to 517753
Data columns (total 15 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id                      517754 non-null  int64  
 1   road_type               517754 non-null  object 
 2   num_lanes               517754 non-null  int64  
 3   curvature               517754 non-null  float64
 4   speed_limit             517754 non-null  int64  
 5   lighting                517754 non-null  object 
 6   weather                 517754 non-null  object 
 7   road_signs_present      517754 non-null  bool   
 8   public_road             517754 non-null  bool   
 9   time_of_day             517754 non-null  object 
 10  holiday                 517754 non-null  bool   
 11  school_season           517754 non-null  bool   
 12  num_reported_accidents  517754 non-null  int64  
 13  accident_risk           517754 non-null  float64
 14  base_risk           

In [8]:
#converting boolean values to int

bool_cols = ['road_signs_present', 'public_road', 'road_signs_present', 'holiday', 'school_season']
for col in bool_cols:
    X_full[f'{col}'] = X_full[f'{col}'].astype(int)
    X_test[f'{col}'] = X_test[f'{col}'].astype(int)

In [9]:
# categorical encoding for categorical features

str_cols = ['road_type', 'lighting','weather', 'time_of_day']
OH = OneHotEncoder(sparse_output = False)
train_oh = pd.DataFrame(OH.fit_transform(X_full[str_cols]))
test_oh = pd.DataFrame(OH.transform(X_test[str_cols]))

X_full = pd.concat([X_full.drop(str_cols, axis = 1), train_oh], axis = 1)
X_test = pd.concat([X_test.drop(str_cols, axis = 1), test_oh], axis = 1)

In [10]:
len(X_full)

517754

In [11]:
X_full = X_full.drop_duplicates()
len(X_full)

517754

**Duplicate values are dropped since they can affect the cross validation process**

In [12]:
# An important step that beginners forget and get error
# All the column name should be of same type before passing to the model

X_full.columns = X_full.columns.astype(str)
X_test.columns = X_test.columns.astype(str)

In [13]:
# Splitting the dataset
X_train, X_val, y_train, y_val = train_test_split(X_full.drop(['id', 'accident_risk'], axis = 1), X_full.accident_risk, test_size = 0.2, random_state = 6)

# XGBoost Regressor

* Device set to CUDA
* GPU enabled

**Increases Speed, earlier the notebook ran in 2-3 mins, now it take less than a minute**

In [14]:
modelx = XGBRegressor(n_estimators = 80, n_jobs = -1, gamma = 0.00001, learning_rate = 0.2, booster = 'gbtree', device = 'cuda', eval_metric = 'rmse', num_parallel_tree = 10,  random_state = 6)
modelx.fit(X_train, y_train)

XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device='cuda', early_stopping_rounds=None,
             enable_categorical=False, eval_metric='rmse', feature_types=None,
             gamma=1e-05, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.2, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=80, n_jobs=-1,
             num_parallel_tree=10, random_state=6, ...)

In [15]:
pred = modelx.predict(X_val)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [15:26:54] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


In [16]:
mean_squared_error(pred, y_val, squared = False)

0.05581162264813006

In [17]:
X_full.drop('id', axis = 1, inplace = True)
test_id = X_test.pop('id')
y = X_full.pop('accident_risk')

In [18]:
X_full.head()

,num_lanes,curvature,speed_limit,road_signs_present,public_road,holiday,school_season,num_reported_accidents,base_risk,0,...,2,3,4,5,6,7,8,9,10,11
0,2,0.06,35,0,1,0,1,1,0.118,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,4,0.99,35,1,0,1,1,0,0.297,0.0,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,4,0.63,70,0,1,1,0,2,0.389,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,4,0.07,35,1,1,0,0,1,0.121,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,1,0.58,60,0,0,1,0,1,0.474,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


**Cross Validation**

In [19]:
score = -1 * cross_val_score(modelx, X_full, y, cv = 5, scoring = 'neg_mean_squared_error')
(score.mean()) ** 0.5

0.05606223539171913

# Final fitting and prediction

In [20]:
modelx.fit(X_full, y)

final_pred = modelx.predict(X_test)

In [21]:
output = pd.DataFrame({'id':test_id, 'accident_risk':final_pred})
output.head()

,id,accident_risk
0,517754,0.295333
1,517755,0.119773
2,517756,0.179758
3,517757,0.310874
4,517758,0.392126


In [22]:
output.to_csv('submission.csv', index = False)